<div style="border-radius:10px;
            border : #015a2c solid;
            background-color:#ecfff5;
           font-size:110%;
           letter-spacing:0.5px;
            text-align: center">

<center><h1 style="padding: 25px 0px; color:#015a2c; font-weight: bold; font-family: Cursive">
🐱 Cats vs. Dogs 🐶</h1></center>
<center><h3 style="padding-bottom: 25px; color:#015a2c; font-weight: bold; font-style:italic; font-family: Cursive">
(CNN model - Image augmentation - Transfer learning)</h3></center>     

</div>

In [ ]:
import os, shutil
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Import Libraries

In [ ]:
import numpy as np 
import pandas as pd 
import zipfile
import matplotlib.pyplot as plt

from tensorflow import keras
from keras import layers
from keras.models import Sequential

from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping

# Define Functions

In [ ]:
def plot_loss_accuracy(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'bo', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training Loss')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

# Import & organize Data

In [ ]:
zip_df = zipfile.ZipFile("/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip", 'r')
zip_df.extractall("/kaggle/working/")
zip_df.close()

In [ ]:
original_dataset_dir = '/kaggle/working/train'
base_dir = '/kaggle/working/catVsdog'
os.mkdir(base_dir)

In [ ]:
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)

# ------------------------------------
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

# ------------------------------------
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)


In [ ]:
print('total training cat images:', len(os.listdir(train_cats_dir)))

In [ ]:
fnames = ['cat.{}.jpg'.format(i) for i in range(10000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

fnames = ['cat.{}.jpg'.format(i) for i in range(10000, 12500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# --------------------------------------------------------------
fnames = ['dog.{}.jpg'.format(i) for i in range(10000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(10000, 12500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)


In [ ]:
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))

# 1. CNN

In [ ]:
model = Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
            optimizer=RMSprop(learning_rate=1e-4),
            metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# --------------------------------------
train_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(150, 150),  
        batch_size=100,
        class_mode='binary')

# --------------------------------------
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=50,
        class_mode='binary')

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=200,  # 20000 train images = batch_size * steps
      epochs=20,
      validation_data=validation_generator,
      validation_steps=100  # 5000 validation images = batch_size * steps
)

In [ ]:
plot_loss_accuracy(history)

<div style="font-family: Cursive; font-size:16px; background-color:#ecfff5; padding: 25px 20px">
The plot above shows the pattern of overfitting.
</div>

# 2. Data augmentation and Dropout layer 

<div style="padding: 10px; font-family: Cursive; border: solid 2px #015a2c;
            font-size:15.5px;padding: 25px 10px; border-radius:8px;">
<p>There are several ways to prevent overfitting, two of which are:</p>
<ol>
    <li>Dropout layer</li>
    <li>Data augmentation</li>
</ol>
</div>

<div style="padding: 10px; font-family: Cursive; border: solid 2px #015a2c;
            font-size:15.5px;padding: 25px 10px; border-radius:8px;">
<h3>What is Dropout? 🤔</h3>
<p>A single model can be used to simulate having a large number of different network architectures by randomly dropping out nodes during training. This is called dropout and offers a very computationally cheap and remarkably effective regularization method to reduce overfitting and improve generalization error in deep neural networks of all kinds. <a href ="https://machinelearningmastery.com/dropout-for-regularizing-deep-neural-networks/">ref</a>
<br><br> The following figure shows a network with dropout.
</p>    
<img src="https://www.researchgate.net/profile/Giorgio-Roffo/publication/317277576/figure/fig23/AS:500357438869504@1496305917227/9-An-illustration-of-the-dropout-mechanism-within-the-proposed-CNN-a-Shows-a.png" alt="dropout"  class="center"> 
<center><a href = " https://www.researchgate.net/figure/9-An-illustration-of-the-dropout-mechanism-within-the-proposed-CNN-a-Shows-a_fig23_317277576">image source</a></center>
</div>

<div style="padding: 10px; font-family: Cursive; border: solid 2px #015a2c;
            font-size:15.5px;padding: 25px 10px; border-radius:8px;">
<h3>What is image data augmentation? 🧐</h3>
<p>image data augmentation is a technique that can be used to artificially expand the size of a training dataset by creating modified versions of images in the dataset.<br> Training deep learning neural network models on more data can result in more skillful models, and the augmentation techniques can create variations of the images that can improve the ability of the fit models to generalize what they have learned to new images. <a href ="https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/">ref</a></p>
<p>Some image augmentation techniques include:</p>
<ul>
    <li>Image shifts</li>    
    <li>Image flips</li>
    <li>Image rotations</li>
    <li>Image zoom</li>
    <li>and ...</li>
</ul>
<p>The following figure shows the augmentation of a cat image.</p>
<img src="https://929687.smushcdn.com/2633864/wp-content/uploads/2021/05/tf_data_data_aug_sequential.png?lossy=1&strip=1&webp=1" alt="augmentation"  class="center" height=100%> 
<center><a href = "https://pyimagesearch.com/2021/06/28/data-augmentation-with-tf-data-and-tensorflow/">image source</a></center>

</div>

In [ ]:
train_datagen_augmentation = ImageDataGenerator(rescale=1./255,
                                                rotation_range=50,
                                                width_shift_range=0.2,
                                                height_shift_range=0.2,
                                                shear_range=0.25,
                                                zoom_range=0.2,
                                                horizontal_flip=True,
                                                fill_mode='nearest')


train_generator = train_datagen_augmentation.flow_from_directory(
        train_dir,  
        target_size=(150, 150),  
        batch_size=100,
        class_mode='binary')

In [ ]:
fname_cat = [os.path.join(train_cats_dir, fname) for fname in os.listdir(train_cats_dir)]

img_path = fname_cat[20]
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)
i = 0
for batch in train_datagen_augmentation.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        break

plt.show()

In [ ]:
fname_dog = [os.path.join(train_dogs_dir, fname) for fname in os.listdir(train_dogs_dir)]

img_path = fname_dog[25]
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)
i = 0
for batch in train_datagen_augmentation.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        break

plt.show()

In [ ]:
model_2 = Sequential()
model_2.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)))
model_2.add(layers.MaxPooling2D(2, 2))
model_2.add(layers.Conv2D(64, (3,3), activation='relu'))
model_2.add(layers.MaxPooling2D(2,2))
model_2.add(layers.Conv2D(128, (3,3), activation='relu'))
model_2.add(layers.MaxPooling2D(2,2))

model_2.add(layers.Flatten())
model_2.add(layers.Dropout(0.5))
model_2.add(layers.Dense(512, activation='relu'))
model_2.add(layers.Dense(1, activation='sigmoid'))

model_2.compile(loss='binary_crossentropy',
            optimizer=RMSprop(learning_rate=1e-4),
            metrics=['accuracy'])


In [ ]:
history = model_2.fit_generator(
      train_generator,
      steps_per_epoch=200,  # 20000 train images = batch_size * steps
      epochs=20,
      validation_data=validation_generator,
      validation_steps=100  # 5000 validation images = batch_size * steps
)

In [ ]:
plot_loss_accuracy(history)

<div style="font-family: Cursive; font-size:16px; background-color:#ecfff5; padding: 25px 20px">
To increase accuracy, we can use transfer learning.
</div>

# 3. Transfer learning

<div style="padding: 10px; font-family: Cursive; border: solid 2px #015a2c;
            font-size:15.5px;padding: 25px 10px; border-radius:8px;">
<h3>What is Transfer Learning?</h3>
<p>Transfer learning is a machine learning technique where a model trained on one task is re-purposed on a second related task. <a href ="https://machinelearningmastery.com/transfer-learning-for-deep-learning/">ref</a> <br> We can use 2 below strategies  for doing transfer learning:</p>
<ol>
    <li>Feature extraction</li>
    <p>Instead of using the model end-to-end as in the previous example, we can treat the pre-trained neural network as a feature extractor by discarding the last fully-connected output layer. This approach allows us to directly apply new dataset to solve an entirely different problem. <a href ="https://towardsdatascience.com/what-is-deep-transfer-learning-and-why-is-it-becoming-so-popular-91acdcc2717a">ref</a></p>
    <li>Fine-tuning</li>
    <p>Unfreeze a few of the top layers of a frozen model base and jointly train both the newly-added classifier layers and the last layers of the base model. This allows us to "fine-tune" the higher-order feature representations in the base model in order to make them more relevant for the specific task. <a href ="https://www.tensorflow.org/tutorials/images/transfer_learning">ref</a></p>
</ol>

</div>

<div style="border-radius:10px;
            background-color:#ffffff;
            border-style: solid;
            border-color: #015a2c;
            letter-spacing:0.5px;">

<center><h3 style="padding: 5px 0px; color:#015a2c; font-weight: bold; font-family: Cursive">
1. Feature extraction</h3></center>
</div>

In [ ]:
base_model = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [ ]:
base_model.summary()

In [ ]:
model_3 = Sequential()
model_3.add(base_model)
model_3.add(layers.Flatten())
model_3.add(layers.Dense(512, activation='relu'))
model_3.add(layers.Dense(1, activation='sigmoid'))

model_3.compile(loss='binary_crossentropy',
            optimizer=RMSprop(learning_rate=1e-4),
            metrics=['accuracy'])

In [ ]:
model_3.summary()

In [ ]:
print('This is the number of trainable weights '
      'before freezing the base model:', len(model_3.trainable_weights))

In [ ]:
base_model.trainable = False

In [ ]:
print('This is the number of trainable weights '
      'after freezing the base model:', len(model.trainable_weights))

In [ ]:
history = model_3.fit_generator(
      train_generator,
      steps_per_epoch=200,  # 20000 train images = batch_size * steps
      epochs=20,
      validation_data=validation_generator,
      validation_steps=100  # 5000 validation images = batch_size * steps
)

In [ ]:
plot_loss_accuracy(history)

<div style="border-radius:10px;
            background-color:#ffffff;
            border-style: solid;
            border-color: #015a2c;
            letter-spacing:0.5px;">

<center><h3 style="padding: 5px 0px; color:#015a2c; font-weight: bold; font-family: Cursive">
2. Fine-Tuning</h3></center>
</div>

In [ ]:
for l in base_model.layers:
    print("layer name is : ",l.name," ** trainable is ", l.trainable)

In [ ]:
base_model.trainable = True
tmp = False
for l in base_model.layers:
    if l.name == "block5_conv1":
        tmp = True
    if tmp:
        l.trainable = True
    else:
        l.trainable = False

In [ ]:
model_3.compile(loss='binary_crossentropy',
            optimizer=RMSprop(learning_rate=1e-4),
            metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(patience=3, monitor='val_loss')

In [ ]:
history = model_3.fit_generator(
      train_generator,
      steps_per_epoch=200,  # 20000 train images = batch_size * steps
      epochs=20,
      validation_data=validation_generator,
      validation_steps=100,  # 5000 validdation images = batch_size * steps
      callbacks=[early_stopping])

In [ ]:
plot_loss_accuracy(history)

# References:
* https://machinelearningmastery.com/dropout-for-regularizing-deep-neural-networks/
* https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/
* https://www.tensorflow.org/tutorials/images/data_augmentation
* https://machinelearningmastery.com/transfer-learning-for-deep-learning/
* https://towardsdatascience.com/what-is-deep-transfer-learning-and-why-is-it-becoming-so-popular-91acdcc2717a
* https://www.tensorflow.org/tutorials/images/transfer_learning

<div style="border-radius:10px;
            background-color:#ffffff;
            border-style: solid;
            border-color: #015a2c;
            letter-spacing:0.5px;">

<center><h4 style="padding: 5px 0px; color:#015a2c; font-weight: bold; font-family: Cursive">
    Thanks for your attention and for reviewing my notebook.🙌 <br><br>Please write your comments for me.📝</h4></center>
<center><h4 style="padding: 5px 0px; color:#015a2c; font-weight: bold; font-family: Cursive">
If you liked my work and found it useful, please upvote. Thank you🙏</h4></center>
</div>